In [21]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [22]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [23]:
subject = 'São Paulo - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [29]:
data = pd.read_csv('../../Inputs Atuais/2008_01_model_input_SP.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
droped_list = ['São Paulo - Produção de Cimento (t)',
               'Direcionamento Aplicação (R$)',
               'Número de Casas',
               'Contratação FGTS (Bi R$)',
               'Poupança e Empréstimo - SBPE (R$)',
               'Ativos Problemáticos - FGTS (R$)',
               'São Paulo - PIB - Per Capita',
               'Quantidade de imóves - Condomínio',
               'Quantidade de imóves – Isolado',
               'Imóvel Área - Privativa (m²)',
               'Global Radiation - Daylight (Kj/m²)']
data = data.drop(droped_list, axis=1)
data

,Ano,São Paulo - IDH Educacao,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Preços de Mercado,São Paulo - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação SFH (Bi R$),...,São Paulo - IDH,São Paulo - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),São Paulo - IDH Longevidade,São Paulo - Consumo de Cimento (t)
0,2008,0.809971,1.092396e+09,4.853082e+07,9.946828e+08,0.794724,5.483335e+08,2.546644e+09,1.874883e+10,3.993366e+10,...,0.818381,8.071263,18.095190,836.334300,15.181901,11.136027,78.961939,1.773094,0.851565,13476.268000
1,2009,0.812524,1.098669e+09,4.873392e+07,9.947096e+08,0.793388,5.302764e+08,2.679945e+09,2.055015e+10,4.439570e+10,...,0.819150,8.100327,17.457437,836.629901,15.173450,11.417327,80.559826,1.537503,0.853009,13304.301000
2,2010,0.814896,1.124010e+09,4.884708e+07,9.985217e+08,0.792439,5.046425e+08,2.891930e+09,2.342247e+10,5.153952e+10,...,0.819803,8.064867,17.476576,836.656415,14.880006,11.288684,81.499868,1.367891,0.853860,14534.980000
3,2011,0.817083,1.146637e+09,4.854680e+07,1.000762e+09,0.791869,4.854622e+08,3.091988e+09,2.604265e+10,5.825126e+10,...,0.820344,8.053175,17.341412,836.557224,14.815765,11.391381,82.321001,1.335640,0.854127,14871.943000
4,2012,0.818822,1.166721e+09,4.823983e+07,1.002188e+09,0.791406,4.867661e+08,3.189506e+09,2.708753e+10,6.141700e+10,...,0.820550,10.058484,16.745399,836.685833,14.895124,11.497643,82.512436,1.341854,0.853708,15159.683000
5,2013,0.821057,1.184231e+09,4.786423e+07,1.003187e+09,0.792190,5.225848e+08,3.093872e+09,2.523394e+10,5.792286e+10,...,0.821262,10.842573,15.620656,836.833718,15.020803,11.542975,82.278650,1.326810,0.852933,15089.297000
6,2014,0.822812,1.197826e+09,4.735435e+07,1.003012e+09,0.792622,5.676413e+08,2.779340e+09,1.954361e+10,4.570163e+10,...,0.821534,11.415800,15.750987,836.986331,15.247907,11.986366,82.848722,1.532951,0.851595,13714.511286
7,2015,0.823629,1.206909e+09,4.689336e+07,1.001922e+09,0.792841,5.179392e+08,2.561731e+09,1.327596e+10,3.539769e+10,...,0.821245,11.865646,15.893940,837.040042,15.520197,12.242333,82.064855,1.539659,0.849644,11767.653429
8,2016,0.832029,1.212819e+09,4.650979e+07,1.001305e+09,0.785642,4.524144e+08,2.516859e+09,1.011370e+10,3.407411e+10,...,0.824327,11.874466,15.955560,837.011044,15.455320,11.972800,81.279861,1.362167,0.857989,9811.084286
9,2017,0.831577,1.214213e+09,4.632537e+07,1.000929e+09,0.785518,4.213689e+08,2.497710e+09,8.456309e+09,3.680756e+10,...,0.823356,11.486716,16.984321,837.006952,15.304181,12.004953,82.751052,1.502384,0.855534,9918.501000


In [30]:
input_data = data.iloc[:-2, 1:-1]
input_data

,São Paulo - IDH Educacao,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Preços de Mercado,São Paulo - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação SFH (Bi R$),Contratação Jurídico Comercial (Bi R$),...,Taxa Selic (%),São Paulo - IDH,São Paulo - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),São Paulo - IDH Longevidade
0,0.809971,1.092396e+09,4.853082e+07,9.946828e+08,0.794724,5.483335e+08,2.546644e+09,1.874883e+10,3.993366e+10,1.825599e+09,...,11.760983,0.818381,8.071263,18.095190,836.334300,15.181901,11.136027,78.961939,1.773094,0.851565
1,0.812524,1.098669e+09,4.873392e+07,9.947096e+08,0.793388,5.302764e+08,2.679945e+09,2.055015e+10,4.439570e+10,2.128973e+09,...,9.472585,0.819150,8.100327,17.457437,836.629901,15.173450,11.417327,80.559826,1.537503,0.853009
2,0.814896,1.124010e+09,4.884708e+07,9.985217e+08,0.792439,5.046425e+08,2.891930e+09,2.342247e+10,5.153952e+10,2.581963e+09,...,9.332485,0.819803,8.064867,17.476576,836.656415,14.880006,11.288684,81.499868,1.367891,0.853860
3,0.817083,1.146637e+09,4.854680e+07,1.000762e+09,0.791869,4.854622e+08,3.091988e+09,2.604265e+10,5.825126e+10,2.924803e+09,...,10.985689,0.820344,8.053175,17.341412,836.557224,14.815765,11.391381,82.321001,1.335640,0.854127
4,0.818822,1.166721e+09,4.823983e+07,1.002188e+09,0.791406,4.867661e+08,3.189506e+09,2.708753e+10,6.141700e+10,2.897722e+09,...,8.159014,0.820550,10.058484,16.745399,836.685833,14.895124,11.497643,82.512436,1.341854,0.853708
5,0.821057,1.184231e+09,4.786423e+07,1.003187e+09,0.792190,5.225848e+08,3.093872e+09,2.523394e+10,5.792286e+10,2.240952e+09,...,7.913561,0.821262,10.842573,15.620656,836.833718,15.020803,11.542975,82.278650,1.326810,0.852933
6,0.822812,1.197826e+09,4.735435e+07,1.003012e+09,0.792622,5.676413e+08,2.779340e+09,1.954361e+10,4.570163e+10,1.011496e+09,...,10.350442,0.821534,11.415800,15.750987,836.986331,15.247907,11.986366,82.848722,1.532951,0.851595
7,0.823629,1.206909e+09,4.689336e+07,1.001922e+09,0.792841,5.179392e+08,2.561731e+09,1.327596e+10,3.539769e+10,7.138599e+08,...,12.454398,0.821245,11.865646,15.893940,837.040042,15.520197,12.242333,82.064855,1.539659,0.849644
8,0.832029,1.212819e+09,4.650979e+07,1.001305e+09,0.785642,4.524144e+08,2.516859e+09,1.011370e+10,3.407411e+10,6.445345e+08,...,13.111908,0.824327,11.874466,15.955560,837.011044,15.455320,11.972800,81.279861,1.362167,0.857989
9,0.831577,1.214213e+09,4.632537e+07,1.000929e+09,0.785518,4.213689e+08,2.497710e+09,8.456309e+09,3.680756e+10,8.272860e+08,...,9.479812,0.823356,11.486716,16.984321,837.006952,15.304181,12.004953,82.751052,1.502384,0.855534


In [31]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     13304.301000
1     14534.980000
2     14871.943000
3     15159.683000
4     15089.297000
5     13714.511286
6     11767.653429
7      9811.084286
8      9918.501000
9     10001.077000
10    10458.276000
11    11181.809000
12    12113.838000
13    12383.649000
14             NaN
Name: São Paulo - Consumo de Cimento (t), dtype: float64

In [32]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,São Paulo - IDH Educacao,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Preços de Mercado,São Paulo - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação SFH (Bi R$),Contratação Jurídico Comercial (Bi R$),...,Taxa Selic (%),São Paulo - IDH,São Paulo - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),São Paulo - IDH Longevidade
0,-1.601914,-1.819408,1.003478,-1.961368,1.285446,1.204752,-0.990234,0.166091,-0.778037,0.174980,...,0.984445,-1.705828,-1.355043,1.636748,-2.157967,0.114751,-1.578154,-2.418314,2.112223,-0.575401
1,-1.272998,-1.675567,1.209427,-1.951541,0.894167,0.820657,-0.546688,0.428911,-0.384215,0.545551,...,-0.051710,-1.234652,-1.336507,0.960381,-0.797175,0.075193,-0.842756,-1.158674,0.402706,0.032912
2,-0.967315,-1.094542,1.324177,-0.558611,0.616212,0.275392,0.158672,0.847995,0.246304,1.098878,...,-0.115145,-0.834215,-1.359122,0.980679,-0.675119,-1.298380,-1.179065,-0.417623,-0.828049,0.391069
3,-0.685507,-0.575726,1.019683,0.259840,0.449046,-0.132596,0.824346,1.230291,0.838686,1.517656,...,0.633403,-0.503056,-1.366579,0.837331,-1.131742,-1.599084,-0.910586,0.229689,-1.062071,0.503752
4,-0.461531,-0.115211,0.708407,0.781023,0.313381,-0.104860,1.148829,1.382743,1.118096,1.484577,...,-0.646476,-0.376604,-0.087675,0.205232,-0.539690,-1.227616,-0.632790,0.380601,-1.016982,0.327130
5,-0.173555,0.286253,0.327534,1.145924,0.543033,0.657046,0.830615,1.112297,0.809701,0.682333,...,-0.757614,0.060098,0.412385,-0.987611,0.141096,-0.639328,-0.514278,0.196304,-1.126146,0.001028
6,0.052526,0.597984,-0.189503,1.082248,0.669744,1.615453,-0.215960,0.282052,-0.268953,-0.819446,...,0.345772,0.226730,0.777966,-0.849388,0.843646,0.423719,0.644870,0.645700,0.369675,-0.562814
7,0.157871,0.806234,-0.656965,0.683738,0.733766,0.558230,-0.940033,-0.632425,-1.178385,-1.183007,...,1.298413,0.049709,1.064859,-0.697781,1.090901,1.698271,1.314042,0.027766,0.418348,-1.384206
8,1.240079,0.941752,-1.045917,0.458458,-1.375468,-0.835563,-1.089341,-1.093812,-1.295205,-1.267688,...,1.596124,1.938197,1.070484,-0.632429,0.957410,1.394591,0.609405,-0.591059,-0.869586,2.130112
9,1.181857,0.973717,-1.232918,0.321024,-1.411648,-1.495939,-1.153056,-1.335633,-1.053949,-1.044458,...,-0.048438,1.343468,0.823193,0.458620,0.938576,0.687130,0.693462,0.568705,0.147869,1.096015


In [33]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0     13304.301000
1     14534.980000
2     14871.943000
3     15159.683000
4     15089.297000
5     13714.511286
6     11767.653429
7      9811.084286
8      9918.501000
9     10001.077000
10    10458.276000
11    11181.809000
Name: São Paulo - Consumo de Cimento (t), dtype: float64

In [34]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,São Paulo - IDH Educacao,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Preços de Mercado,São Paulo - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação SFH (Bi R$),Contratação Jurídico Comercial (Bi R$),...,Taxa Selic (%),São Paulo - IDH,São Paulo - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),São Paulo - IDH Longevidade
12,1.001135,0.409525,-0.926633,-1.033191,-1.385738,1.108405,2.635823,-0.189959,2.668333,-0.032632,...,-2.260447,-1.092902,0.514165,-0.901047,-0.751547,-0.363994,-0.503159,-0.998268,-1.560012,-2.468148


In [35]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    12113.838
Name: São Paulo - Consumo de Cimento (t), dtype: float64

In [36]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-1:], train_target.iloc[-1:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [37]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [38]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[3337341868, 1406950038, 2346113991, 3171828438, 2082289352, 3240228219, 417464784, 2565830355, 3909558316, 3353971994, 4157878872, 2565453597, 3561148247, 3245365631, 3721326983, 134457545, 1779715043, 41263096, 3508863208, 2435559849, 1461997159, 1053367341, 1451479955, 3706304883, 1737008401, 1818793450, 3384741538, 580592576, 1678619843, 4128757998, 2897325606, 539834976, 3143289062, 711551101, 329583058, 3492304111, 628785925, 3358203268, 1452274896, 1121801012, 1020119263, 3870566566, 3525005623, 60787249, 55267132, 1892611604, 3020999409, 1156854330, 2729567274, 1171717257, 892347008, 2819004127, 2579228580, 2028708984, 4258886200, 1366388523, 574744159, 4005459162, 2726958190, 2183240643, 2059483598, 2594075150, 3381783160, 311655240, 2125584914, 4276584973, 1868876473, 2427353146, 1851901632, 837423848, 1406239515, 70196679, 2012826601, 4077382530, 2883727975, 1080998223, 2209063287, 1873257782, 74160871, 2315821734, 154657773, 1642632368, 622856446, 2989164891, 1119305726, 42

val_loss: 8136.0751953125


Step: 89 ___________________________________________
val_loss: 93938.65625


Step: 90 ___________________________________________
val_loss: 104589.078125


Step: 91 ___________________________________________
val_loss: 263481.6875


Step: 92 ___________________________________________
val_loss: 154301.65625


Step: 93 ___________________________________________
val_loss: 434003.03125


Step: 94 ___________________________________________
val_loss: 325101.53125


Step: 95 ___________________________________________
val_loss: 358792.8125


Step: 96 ___________________________________________
val_loss: 317085.5625


Step: 97 ___________________________________________
val_loss: 0.3992166519165039
winner_seed: 3997733236


Step: 98 ___________________________________________
val_loss: 370082.125


Step: 99 ___________________________________________
val_loss: 302409.78125


final_seed: 3997733236


In [39]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 483ms/step - loss: 160207376.0000 - val_loss: 130180904.0000
Epoch 2/10000
1/1 [==============================] - 0s 14ms/step - loss: 153972240.0000 - val_loss: 34870392.0000
Epoch 3/10000
1/1 [==============================] - 0s 13ms/step - loss: 146476496.0000 - val_loss: 65230592.0000
Epoch 4/10000
1/1 [==============================] - 0s 13ms/step - loss: 163164368.0000 - val_loss: 131289792.0000
Epoch 5/10000
1/1 [==============================] - 0s 13ms/step - loss: 152625552.0000 - val_loss: 73506880.0000
Epoch 6/10000
1/1 [==============================] - 0s 13ms/step - loss: 132385496.0000 - val_loss: 197984768.0000
Epoch 7/10000
1/1 [==============================] - 0s 13ms/step - loss: 132725816.0000 - val_loss: 163260144.0000
Epoch 8/10000
1/1 [==============================] - 0s 13ms/step - loss: 128837336.0000 - val_loss: 163465072.0000
Epoch 9/10000
1/1 [==============================] - 0s 13ms/step - loss: 

Epoch 72/10000
1/1 [==============================] - 0s 14ms/step - loss: 37855308.0000 - val_loss: 69951072.0000
Epoch 73/10000
1/1 [==============================] - 0s 14ms/step - loss: 32471056.0000 - val_loss: 77618616.0000
Epoch 74/10000
1/1 [==============================] - 0s 13ms/step - loss: 35946208.0000 - val_loss: 64164764.0000
Epoch 75/10000
1/1 [==============================] - 0s 13ms/step - loss: 34276596.0000 - val_loss: 73509024.0000
Epoch 76/10000
1/1 [==============================] - 0s 13ms/step - loss: 33273130.0000 - val_loss: 89187048.0000
Epoch 77/10000
1/1 [==============================] - 0s 13ms/step - loss: 28550770.0000 - val_loss: 61699468.0000
Epoch 78/10000
1/1 [==============================] - 0s 13ms/step - loss: 29679728.0000 - val_loss: 32847282.0000
Epoch 79/10000
1/1 [==============================] - 0s 13ms/step - loss: 24807576.0000 - val_loss: 55791780.0000
Epoch 80/10000
1/1 [==============================] - 0s 13ms/step - loss: 31340

Epoch 143/10000
1/1 [==============================] - 0s 13ms/step - loss: 14521269.0000 - val_loss: 13662769.0000
Epoch 144/10000
1/1 [==============================] - 0s 13ms/step - loss: 10761330.0000 - val_loss: 9394180.0000
Epoch 145/10000
1/1 [==============================] - 0s 13ms/step - loss: 5630628.0000 - val_loss: 20989686.0000
Epoch 146/10000
1/1 [==============================] - 0s 13ms/step - loss: 13197952.0000 - val_loss: 10772165.0000
Epoch 147/10000
1/1 [==============================] - 0s 13ms/step - loss: 15038599.0000 - val_loss: 8366290.5000
Epoch 148/10000
1/1 [==============================] - 0s 13ms/step - loss: 7910045.5000 - val_loss: 16095576.0000
Epoch 149/10000
1/1 [==============================] - 0s 13ms/step - loss: 13140480.0000 - val_loss: 18676864.0000
Epoch 150/10000
1/1 [==============================] - 0s 13ms/step - loss: 13103211.0000 - val_loss: 7995816.5000
Epoch 151/10000
1/1 [==============================] - 0s 13ms/step - loss: 1

1/1 [==============================] - 0s 14ms/step - loss: 3546421.0000 - val_loss: 30630252.0000
Epoch 215/10000
1/1 [==============================] - 0s 14ms/step - loss: 3421782.0000 - val_loss: 23208476.0000
Epoch 216/10000
1/1 [==============================] - 0s 13ms/step - loss: 4325514.5000 - val_loss: 4777315.0000
Epoch 217/10000
1/1 [==============================] - 0s 14ms/step - loss: 5159129.0000 - val_loss: 6475240.5000
Epoch 218/10000
1/1 [==============================] - 0s 13ms/step - loss: 2903696.7500 - val_loss: 6045600.5000
Epoch 219/10000
1/1 [==============================] - 0s 13ms/step - loss: 4843662.5000 - val_loss: 5997122.5000
Epoch 220/10000
1/1 [==============================] - 0s 13ms/step - loss: 2317955.2500 - val_loss: 6169984.5000
Epoch 221/10000
1/1 [==============================] - 0s 14ms/step - loss: 4627440.5000 - val_loss: 5660877.0000
Epoch 222/10000
1/1 [==============================] - 0s 13ms/step - loss: 5621786.0000 - val_loss: 5

1/1 [==============================] - 0s 14ms/step - loss: 2882490.7500 - val_loss: 2309203.7500
Epoch 287/10000
1/1 [==============================] - 0s 14ms/step - loss: 3429994.7500 - val_loss: 2615617.5000
Epoch 288/10000
1/1 [==============================] - 0s 14ms/step - loss: 1267753.3750 - val_loss: 2095923.5000
Epoch 289/10000
1/1 [==============================] - 0s 15ms/step - loss: 1280766.1250 - val_loss: 2936952.5000
Epoch 290/10000
1/1 [==============================] - 0s 15ms/step - loss: 2691192.2500 - val_loss: 1713164.0000
Epoch 291/10000
1/1 [==============================] - 0s 15ms/step - loss: 2619489.2500 - val_loss: 3610486.2500
Epoch 292/10000
1/1 [==============================] - 0s 14ms/step - loss: 4198077.5000 - val_loss: 1672417.2500
Epoch 293/10000
1/1 [==============================] - 0s 14ms/step - loss: 3135108.0000 - val_loss: 2749330.0000
Epoch 294/10000
1/1 [==============================] - 0s 14ms/step - loss: 1671748.3750 - val_loss: 200

...

Epoch 1311/10000
1/1 [==============================] - 0s 13ms/step - loss: 311806.3750 - val_loss: 62066.1836
Epoch 1312/10000
1/1 [==============================] - 0s 13ms/step - loss: 286444.8750 - val_loss: 48997.8125
Epoch 1313/10000
1/1 [==============================] - 0s 13ms/step - loss: 362749.4688 - val_loss: 27970.2754
Epoch 1314/10000
1/1 [==============================] - 0s 13ms/step - loss: 295725.0938 - val_loss: 90208.1250
Epoch 1315/10000
1/1 [==============================] - 0s 13ms/step - loss: 260308.3750 - val_loss: 107127.0781
Epoch 1316/10000
1/1 [==============================] - 0s 13ms/step - loss: 337985.1250 - val_loss: 32823.6016
Epoch 1317/10000
1/1 [==============================] - 0s 13ms/step - loss: 334584.1562 - val_loss: 38561.6055
Epoch 1318/10000
1/1 [==============================] - 0s 13ms/step - loss: 281270.8438 - val_loss: 96338.7031
Epoch 1319/10000
1/1 [==============================] - 0s 13ms/step - loss: 345767.2812 - val_loss: 50

1/1 [==============================] - 0s 13ms/step - loss: 289182.7500 - val_loss: 32442.5527
Epoch 1385/10000
1/1 [==============================] - 0s 13ms/step - loss: 302946.0312 - val_loss: 37319.5234
Epoch 1386/10000
1/1 [==============================] - 0s 13ms/step - loss: 319396.3125 - val_loss: 28341.5918
Epoch 1387/10000
1/1 [==============================] - 0s 13ms/step - loss: 279946.2188 - val_loss: 19697.4648
Epoch 1388/10000
1/1 [==============================] - 0s 14ms/step - loss: 248617.0781 - val_loss: 63970.4609
Epoch 1389/10000
1/1 [==============================] - 0s 14ms/step - loss: 337607.4688 - val_loss: 31905.4961
Epoch 1390/10000
1/1 [==============================] - 0s 14ms/step - loss: 303880.7812 - val_loss: 76725.7578
Epoch 1391/10000
1/1 [==============================] - 0s 14ms/step - loss: 263228.7500 - val_loss: 35949.4922
Epoch 1392/10000
1/1 [==============================] - 0s 14ms/step - loss: 252055.5469 - val_loss: 66435.5625
Epoch 139

1/1 [==============================] - 0s 14ms/step - loss: 340918.3438 - val_loss: 20751.1895
Epoch 1458/10000
1/1 [==============================] - 0s 14ms/step - loss: 257668.2031 - val_loss: 63997.1406
Epoch 1459/10000
1/1 [==============================] - 0s 14ms/step - loss: 291097.5000 - val_loss: 17958.0938
Epoch 1460/10000
1/1 [==============================] - 0s 14ms/step - loss: 329803.3438 - val_loss: 51932.3555
Epoch 1461/10000
1/1 [==============================] - 0s 14ms/step - loss: 347192.9062 - val_loss: 47317.2969
Epoch 1462/10000
1/1 [==============================] - 0s 14ms/step - loss: 265853.5625 - val_loss: 69220.3750
Epoch 1463/10000
1/1 [==============================] - 0s 14ms/step - loss: 308288.7812 - val_loss: 11351.4043
Epoch 1464/10000
1/1 [==============================] - 0s 14ms/step - loss: 303691.2812 - val_loss: 51011.5742
Epoch 1465/10000
1/1 [==============================] - 0s 14ms/step - loss: 279101.5938 - val_loss: 38389.2070
Epoch 146

1/1 [==============================] - 0s 15ms/step - loss: 240206.2500 - val_loss: 42812.6211
Epoch 1531/10000
1/1 [==============================] - 0s 14ms/step - loss: 206212.8750 - val_loss: 86047.1172
Epoch 1532/10000
1/1 [==============================] - 0s 14ms/step - loss: 273892.2500 - val_loss: 18647.1836
Epoch 1533/10000
1/1 [==============================] - 0s 15ms/step - loss: 374056.0000 - val_loss: 11260.4434
Epoch 1534/10000
1/1 [==============================] - 0s 15ms/step - loss: 287101.1875 - val_loss: 27033.5898
Epoch 1535/10000
1/1 [==============================] - 0s 15ms/step - loss: 280796.6250 - val_loss: 16113.6250
Epoch 1536/10000
1/1 [==============================] - 0s 15ms/step - loss: 325458.9688 - val_loss: 11175.8350
Epoch 1537/10000
1/1 [==============================] - 0s 15ms/step - loss: 382998.5938 - val_loss: 20741.9062
Epoch 1538/10000
1/1 [==============================] - 0s 15ms/step - loss: 251477.5625 - val_loss: 30921.3672
Epoch 153

1/1 [==============================] - 0s 13ms/step - loss: 343085.9688 - val_loss: 11290.7227
Epoch 1604/10000
1/1 [==============================] - 0s 13ms/step - loss: 214674.6719 - val_loss: 80749.7656
Epoch 1605/10000
1/1 [==============================] - 0s 14ms/step - loss: 364239.0938 - val_loss: 239.7659
Epoch 1606/10000
1/1 [==============================] - 0s 13ms/step - loss: 225585.6406 - val_loss: 61107.4531
Epoch 1607/10000
1/1 [==============================] - 0s 13ms/step - loss: 258171.7500 - val_loss: 68502.8359
Epoch 1608/10000
1/1 [==============================] - 0s 13ms/step - loss: 286707.9062 - val_loss: 9667.4600
Epoch 1609/10000
1/1 [==============================] - 0s 13ms/step - loss: 295094.2188 - val_loss: 20017.5527
Epoch 1610/10000
1/1 [==============================] - 0s 13ms/step - loss: 296976.5312 - val_loss: 22252.5391
Epoch 1611/10000
1/1 [==============================] - 0s 14ms/step - loss: 267766.5625 - val_loss: 21580.0117
Epoch 1612/1

In [40]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 54ms/step
[12113.838](test_target) - [[11182.44]](prediction) = 931.3975703124997


In [41]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [42]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,São Paulo - IDH Educacao,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Preços de Mercado,São Paulo - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação SFH (Bi R$),Contratação Jurídico Comercial (Bi R$),...,Taxa Selic (%),São Paulo - IDH,São Paulo - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),São Paulo - IDH Longevidade
0,-1.239414,-0.924099,-1.323126,-0.714550,1.287736,1.148205,-1.122043,-1.121418,-1.120483,-1.136601,...,1.412346,-1.256499,-0.489673,1.413722,-1.410322,0.736997,-1.258070,-1.315911,1.285551,-1.315803
1,0.029885,-0.465059,0.229123,-0.699637,-0.137605,0.140893,-0.184474,-0.185492,-0.187012,-0.160469,...,-0.643250,0.066193,1.393821,-0.739164,0.614359,0.676789,1.188450,0.209310,-0.132366,0.209017
2,1.209529,1.389158,1.094002,1.414187,-1.150131,-1.289098,1.306518,1.306910,1.307495,1.297069,...,-0.769096,1.190306,-0.904148,-0.674558,0.795962,-1.413786,0.069619,1.106602,-1.153185,1.106786


train_target:


,São Paulo - Consumo de Cimento (t)
0,13304.301
1,14534.980
2,14871.943


test_input:


...

,São Paulo - IDH Educacao,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Preços de Mercado,São Paulo - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação SFH (Bi R$),Contratação Jurídico Comercial (Bi R$),...,Taxa Selic (%),São Paulo - IDH,São Paulo - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),São Paulo - IDH Longevidade
12,1.001135,0.409525,-0.926633,-1.033191,-1.385738,1.108405,2.635823,-0.189959,2.668333,-0.032632,...,-2.260447,-1.092902,0.514165,-0.901047,-0.751547,-0.363994,-0.503159,-0.998268,-1.560012,-2.468148


test_target:


,São Paulo - Consumo de Cimento (t)
12,12113.838


1/1 [==============================] - 0s 34ms/step
Error: 931.3975703124997




[14871.6533203125,
 15159.8115234375,
 15040.2685546875,
 11911.8232421875,
 11767.6484375,
 10280.240234375,
 10234.630859375,
 10254.884765625,
 10438.9287109375,
 11182.4404296875]

In [43]:
display(targets)
display(predictions)

[15159.683,
 15089.297,
 13714.511285714289,
 11767.65342857143,
 9811.084285714289,
 9918.501,
 10001.077,
 10458.276,
 11181.809,
 12113.838]

[14871.6533203125,
 15159.8115234375,
 15040.2685546875,
 11911.8232421875,
 11767.6484375,
 10280.240234375,
 10234.630859375,
 10254.884765625,
 10438.9287109375,
 11182.4404296875]

In [44]:
mae = mean_absolute_error(predictions, targets)
mae

625.7997624999992

In [45]:
porcentage = mae/np.mean(targets)
porcentage

0.052493052930179536